In [1]:
import os
import cv2
import json
import random
import logging
import numpy as np
import pandas as pd
from tensorflow.keras.utils import load_img
import tensorflow as tf

import exportTool as export

from IPython.display import clear_output

clear_output()

In [2]:
# video_number = '010'

In [3]:
def compute_error(y, y_pred):
    """
    Since the competion computes the error based on Euclidian distances, this function computes
    the error for predicted and ground truth results.
    
    @param: list y. a (1, 2) list with original coordinations.
    @param: list y_pred. a (1, 2) list containing the predicted coordinations.
    
    @return: float error. The Euclidian distance between these two given datapoints.
    """
    error = np.sqrt(np.abs(y[0] - y_pred[0]) ** 2 + np.abs(y[1] - y_pred[1]) ** 2)
    return error

In [4]:
def compute_original_coordination(predicted_coordination, original_box_coordination):
    """
    Computes the coordinations of the predicted Hexbug's head's coordination in the original image
    using the bounding box coordinations.
    
    @param: list predicted_coordination. a (1, 2) list with (x, y) format.
    @param: list original_box_coordination. a (1, 2) list with (x, y) format of bounding box.
    
    @return: list. Coordinations of the predicted Hexbug's head's coordination in the original image
    """
    return (predicted_coordination[0] + original_box_coordination[0], predicted_coordination[1] + original_box_coordination[1])

In [5]:
reconstructed_model = tf.keras.models.load_model("resnet50_trained_model_data/v1.5/checkpoint_callback/")
clear_output()

In [6]:
data = pd.read_csv('Test_data/data_multi_object.csv').drop(['Unnamed: 0'], axis=1).sort_values(['Path', 'HexBugID'], ascending=True)
data.head(5)

,NumHexBugsInPicture,HexBugID,OriginalBoxCoordinationX1,OriginalBoxCoordinationY1,Path,ID
376,2,0,280,838,Test_data/cropped_bugs_multi_object/001/frame0...,0
375,2,1,742,107,Test_data/cropped_bugs_multi_object/001/frame0...,0
312,3,0,728,208,Test_data/cropped_bugs_multi_object/001/frame1...,1
311,3,1,172,931,Test_data/cropped_bugs_multi_object/001/frame1...,1
310,3,2,210,112,Test_data/cropped_bugs_multi_object/001/frame1...,1


In [7]:
os.mkdir(f'Test_data/annotated_images_multi_object/')

for i in range(1, 6):
    os.mkdir(f'Test_data/annotated_images_multi_object/00{i}')

In [8]:
for idx, row in data.iterrows():
    img_path = row.Path.replace('.jpg', f'_{row.HexBugID}.jpg')
    cropped_img = load_img(img_path)
    np_cropped_img = np.array(cropped_img)
    
    expanded_img = np.expand_dims(np_cropped_img, 0)
    predicted = list(reconstructed_model.predict(expanded_img)[0])
    
    scaled_coordinations = list(compute_original_coordination(
            predicted, (row.OriginalBoxCoordinationX1,
            row.OriginalBoxCoordinationY1)))
    
    int_scaled_predicted = list(map(int, scaled_coordinations))
    
    if os.path.exists(row.Path.replace('cropped_bugs_multi_object', 'annotated_images_multi_object')):
        original_image_path = row.Path.replace('cropped_bugs_multi_object', 'annotated_images_multi_object')
    else:
        original_image_path = row.Path.replace('cropped_bugs_multi_object', 'Frames')
    
    original_sample_img = np.array(load_img(original_image_path))
    
    radius = 15
    
    if row.HexBugID == 0:
        color = (255, 0, 0)
    elif row.HexBugID == 1:
        color = (0, 255, 0)
    elif row.HexBugID == 2:
        color = (0, 0, 255)
    elif row.HexBugID == 3:
        color = (0, 255, 255)
    else:
        color = (255, 0, 255)
    
    thickness = -1
    
    cv2.circle(original_sample_img, int_scaled_predicted, radius, color, thickness)
    cv2.imwrite(row.Path.replace('cropped_bugs_multi_object', 'annotated_images_multi_object'), original_sample_img)

2023-05-11 14:29:42.387244: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600


1/1 [==============================] - 2s 2s/step


2023-05-11 14:29:42.816032: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 19ms/step


In [9]:
# tmp = [] # you can skip this step
# for val in result_array: 
#     tmp = export.fromArrayToDict(val,tmp) #  fromArrayToDict(val)

# export.saveList(tmp, f"Test_data/test{directory_name}.csv")